In [ ]:
### import libraries

import pandas as pd
import numpy as np
import openpyxl as opx
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

In [ ]:
### input variables

mainPath = '' # path to all files

allLocusTagsFilename = '' # file name (excel) of the locus tags to be checked
sheetName = '' # sheet name if exist; optional parameter
diffExprColName = '' # name of the column with information about the sign of differential expression

cytokin = '' # name of cytokin; 'TNF' and 'IL6' were used in this work

orthogroupsFilename = '' # name of the table with orthogroups from the Orthofinder outputs; for example, 'Orthogroups.tsv'
orthogroupsGeneCountFilename = '' # name of the table with gene counts from the Orthofinder outputs; for example, 'Orthogroups.GeneCount.tsv'
organismName = '' # name of the analysed organism which is the same as in the tables from the Orthofinder outputs; for example, 'GCF_000772485.1_ASM77248v1_translated_cds'
organismFaaFilename = '' # file name (fasta; translated cds) of the organism to be analysed

minimalOGdist = 0.01 # the minimum value of the threshold for recognition of 'neighboring' orthogroups
genomeDist = 10000 # the minimum distance in genome of the analysed organism for searching connected genes

In [ ]:
### opening file with locus tags to be checked

if sheetName != '':
    allLocusTags_df = pd.read_excel(io = mainPath + allLocusTagsFilename, sheet_name=sheetName)
else:
    allLocusTags_df = pd.read_excel(io = mainPath + allLocusTagsFilename)

# checking
allLocusTags_df.head()

In [ ]:
### opening file (fasta; translated cds) of the organism to be analysed

file = open(mainPath + organismFaaFilename, 'r')
organismFaaFile = file.read()
file.close()

# checking
organismFaaFile

In [ ]:
### function for finding WP-number, location and sign of strain of protein in organismFaaFile using locus 
### tags from allLocusTags_df

def get_WP_and_location(tag, text):
    
    tag_str_index = text.find('locus_tag' + '=' + tag)
    
    if tag_str_index == -1:
        WP, loc, strain = '', '', ''

    else:
        str_with_WP_and_loc = text[tag_str_index - 40:tag_str_index + 400]
        WP_str_index = str_with_WP_and_loc.find('_prot_')
        WP = str_with_WP_and_loc[WP_str_index + 1 : WP_str_index + 20]
        if WP[6:10] != 'WP_':
            WP = WP.split(' ')[0]  
        loc_str_index = str_with_WP_and_loc.find('[location=')
        loc_str_index_end = str_with_WP_and_loc.find('] [gbkey=')
        loc = str_with_WP_and_loc[loc_str_index + 10 : loc_str_index_end]
        if loc[:3] == 'com':
            strain = '-'
        else:
            strain = '+'
    
    return(WP, loc, strain)

# checking
get_WP_and_location('BLGT_RS00065', organismFaaFile)

In [ ]:
### function for creating data frame from dictionary with locus tags from primary file and received WP, 
### location and sign of strain using function get_WP_and_location

def create_tags_WP_df(tags_lst, text):
    
    WP_lst, loc_lst, strain_lst = [], [], []
    
    for tag in tags_lst:
        tmp = get_WP_and_location(tag, text)
        WP_lst.append(tmp[0])
        loc_lst.append(tmp[1])
        strain_lst.append(tmp[2])
        
    tags_WP_dic = { 'locus_tag': tags_lst, 'protein': WP_lst, 'location': loc_lst, 'strain': strain_lst }
    
    return pd.DataFrame.from_dict(tags_WP_dic)

In [ ]:
### creating data frame tagsProtein_df using function create_tags_WP_df and adding to this data frame 
### data with information about differential expression

allLocusTags_lst = allLocusTags_df.locus_tag.tolist()
tagsProtein_df = create_tags_WP_df(allLocusTags_lst, organismFaaFile)
tagsProtein_df = tagsProtein_df.merge(allLocusTags_df[['locus_tag', diffExprColName]], how='inner')

# checking
tagsProtein_df

In [ ]:
### opening file with orthogroups (OG) and WPs from Orthofinder's outputs

orthogroups_df = pd.read_csv(mainPath + orthogroupsFilename, sep='\t', encoding='UTF-8')
orthogroups_df = orthogroups_df.replace(np.nan, '', regex=True)

# checking
orthogroups_df

In [ ]:
### creating function getting respective orthogroups  

def get_OG(orthogroups_df, organism_name, tagsWP_df):
    
    col = organism_name
    WP_lst, OG_ind_lst, OG_lst = [], [], []
    
    for sub in tagsWP_df.protein.to_list():
        
        if sub != '':
            
            found = pd.DataFrame()
            found[col] = orthogroups_df[col].str.contains(sub, regex=False)
            found_subset = found[col].loc[found[col] > 0]
            
            if not (found_subset.empty):
                
                WP_lst.append(sub)
                ind = found_subset.index[0]
                OG_ind_lst.append(ind)
                OG_lst.append(str(orthogroups_df.iloc[ind, 0]))
                
    WP_OG = pd.DataFrame.from_dict({ 'protein': WP_lst, 'OG_index': OG_ind_lst, 'OG': OG_lst })
    
    return(WP_OG)

In [ ]:
### inner joining of the tagsProteinOG_df and tagsProtein_df data frames by "WP" key

tagsProteinOG_df = get_OG(orthogroups_df, organismName, tagsProtein_df)
tagsProtein_df = tagsProtein_df.merge(tagsProteinOG_df, how='inner')

# checking
tagsProtein_df.tail()

In [ ]:
### creating data frame with orthogroups and gene counts from Orthofinder's outputs and removing 'Total' 
### column from gene_count data frame

geneCount_df = pd.read_csv(mainPath + orthogroupsGeneCountFilename, sep='\t', encoding='UTF-8')
geneCount_df.drop(['Total'], axis=1, inplace=True)

# checking
geneCount_df

In [ ]:
### converting strings to numeric and binary vectors

for i in range(1, len(geneCount_df.iloc[0, :])):
    pd.to_numeric(geneCount_df.iloc[:, i])

col_names = geneCount_df.columns.to_list()[1:]
for col in col_names:
    geneCount_df.loc[geneCount_df[col] > 0, col] = 1 
    
# checking 
geneCount_df.head()

In [ ]:
### geneCount_df is a data frame of an m by n array of m original observations in an n-dimensional space

geneCount_df = geneCount_df.drop(['Orthogroup'], axis=1)

# checking
geneCount_df.head()

In [ ]:
### creating jaccard-pdist upper-triangle matrix of distances and converting it to squared matrix of difference

distMatrix = np.around(pdist(geneCount_df, 'jaccard'), 3)
distMatrix = squareform(distMatrix)

# checking
distMatrix

In [ ]:
### changing diaganal values '0' to '1' in distMatrix and creating data frame with distances

np.fill_diagonal(distMatrix, 1)
distMatrix_df = pd.DataFrame(data=distMatrix)

# checking
distMatrix_df

In [ ]:
# checking
distMatrix_df.loc[distMatrix_df[9] <= minimalOGdist] #column with number OG_index, where distance <= minimalOGdist

In [ ]:
### finding other OG with minimal distances from considered OG* (OG <= minimalOGdist) and creating dictionary,
### where considered OG* is a key and found OGs are values

def find_min_OG_dist(tagsProtein_df, distMatrix_df, minimalOGdist):
    
    dict_with_min_distances_OG_to_df = {}
    dict_with_min_distances_OG = {}

    for OG_index in tagsProtein_df.OG_index.to_list():
        
        tmp = distMatrix_df.loc[distMatrix_df[OG_index] <= minimalOGdist] #column with number OG_index, where distance <= minimalOGdist
        
        if not tmp.empty:
        
            dict_with_min_distances_OG.update({OG_index : tmp.index.to_list()})

            dict_with_min_distances_OG_to_df.update({OG_index : ', '.join(map(str, tmp.index.to_list()))})
            
    return(dict_with_min_distances_OG, dict_with_min_distances_OG_to_df)

In [ ]:
### creating data frame of found OG index with minimal distances

minOGDist_dict = find_min_OG_dist(tagsProtein_df, distMatrix_df, minimalOGdist)

minOGDist_df = pd.DataFrame(minOGDist_dict[1], index=['found_OG_index_with_min_dist']).T
minOGDist_df['OG_index'] = minOGDist_df.index

# checking
minOGDist_df

In [ ]:
### left joining of data frames tagsProtein_df and OG_with_min_dist_df by "OG_index" key

tagsProtein_df = tagsProtein_df.merge(minOGDist_df, on='OG_index', how='left')

# checking
tagsProtein_df

In [ ]:
### searching for every found OG with minimal distances from considered OG* (OG <= minimalOGdist) respective WPs

foundProtein_dict = {}
minOGDist_df_len = len(minOGDist_df)
n = 1

for key in minOGDist_dict[0].keys():
    print('Checking: ' + str(n) + ' OG index out of ' + str(minOGDist_df_len))
    for val in minOGDist_dict[0][key]: 
        lst_tmp = orthogroups_df.loc[val, [organismName]].to_list()
        
        if lst_tmp != ['']:
            foundProtein_dict.update({ val : lst_tmp })
    n += 1

In [ ]:
### creating data frame of found OG indexes with minimal distances and their found WPs (protein)

foundProtein_df = pd.DataFrame(foundProtein_dict, index=['found_protein']).T
foundProtein_df['OG_index'] = foundProtein_df.index


# checking
foundProtein_df
#foundProtein_df.loc[foundProtein_df['OG_index'] == 5].found_protein.values

In [ ]:
### division data frame "found_protein_df" into separate WP

found_protein = foundProtein_df.pop('found_protein').str.split(', ', expand=True)\
.stack().reset_index(1, drop=True).rename('found_protein')

foundProtein_df = foundProtein_df.join(found_protein).reset_index(drop=True)

# checking
foundProtein_df

In [ ]:
### searching location for every protein (WP) from data frame "found_protein_df" in longum_cds.txt

loc_lst, strain_lst = [], []
prot_lst = foundProtein_df.found_protein.to_list() 

for prot in prot_lst:
    
    prot = prot.split()[0]       
    prot_str_index = organismFaaFile.find(prot)
    
    if prot_str_index == -1:
        
        loc, strain = '', ''
        
    else:
        
        str_with_loc = organismFaaFile[prot_str_index:prot_str_index + 400]
        loc_str_index = str_with_loc.find('[location=')
        loc_str_index_end = str_with_loc.find('] [gbkey=')
        loc = str_with_loc[loc_str_index + 10:loc_str_index_end]
        
        if loc[:3] == 'com':
            strain = '-'
        else:
            strain = '+'
        
    loc_lst.append(loc)
    strain_lst.append(strain)
    
foundProteinLoc_df = pd.DataFrame({ 'found_protein': prot_lst , 'location': loc_lst, 'strain': strain_lst })

# checking
foundProteinLoc_df.head()

In [ ]:
### inner joining of data frames tagsWPdf and OG_with_min_dist_df by "OG_index" key

foundProteinLoc_df = foundProtein_df.merge(foundProteinLoc_df, on='found_protein', how='inner')

# checking
foundProteinLoc_df.head(30)

In [ ]:
### division data frame "found_OG_index_with_min_dist" into separate OG

found_OG_index_with_min_dist = minOGDist_df.pop('found_OG_index_with_min_dist').str.split(',', expand=True)\
.stack().reset_index(1, drop=True).rename('found_OG_index_with_min_dist')

minOGDist_df = minOGDist_df.join(found_OG_index_with_min_dist).reset_index(drop=True)

# checking
minOGDist_df

In [ ]:
### removing empty values of found_OG_index_with_min_dist column

tagsProtein_df.replace(np.nan, '', inplace = True)
tagsProtein_df = tagsProtein_df.loc[tagsProtein_df['found_OG_index_with_min_dist'] != '']

# checking 
tagsProtein_df

In [ ]:
### searching for every initial locus tags found WPs with minimal distances and nearby location (<=genomeDist);
### getting all possible suitable WPs

locusFoundProtein_dict = {}

ind_lst = tagsProtein_df.index
n = 1
for i in ind_lst:
    
    print('Checking: ' + str(n) + ' index out of ' + str(len(ind_lst)))
    
    loc_val = []
    #list with found OG_index with minimal distances (OG <= 0.005)
    lst_OG = tagsProtein_df.loc[i, ['found_OG_index_with_min_dist']].to_list()[0].split(', ')
    
    if lst_OG == ['']:
        continue
        
    #initial location 
    compare_loc = tagsProtein_df.loc[i, ['location']].values[0].split('..')
    compare_strain = tagsProtein_df.loc[i, ['strain']].values[0]
    
    compare_loc_start = compare_loc[0].replace('<', '')
    if compare_loc_start[:3] == 'com':
        compare_loc_start = compare_loc_start.split('(')[1]

    compare_loc_end = compare_loc[1].replace('>', '').replace(')', '')
    
    dict_loc_val_ind = {}
    
    for item in lst_OG:
        
        loc_subset = foundProteinLoc_df.loc[foundProteinLoc_df['OG_index'] == int(item)]
        loc_indexes = loc_subset.index
        
        for k in range(0, len(loc_indexes)):

            loc_val = loc_subset.location.values[k].split('..')
            if not loc_val:
                continue
        
            loc_val_start = loc_val[0].replace('<', '')
            if loc_val_start[:3] == 'com':
                loc_val_start = loc_val_start.split('(')[1]
    
            loc_val_end = loc_val[1].replace('>', '').replace(')', '')
                
            loc_index = loc_indexes[k]
            strain_val = loc_subset.strain.values[k]
                    
            dict_loc_val_ind.update({ (int(loc_val_start), int(loc_val_end), strain_val) : loc_index })
        
    lst_found_protein = []
    for val, ind in dict_loc_val_ind.items():
        
        start_cond = int(compare_loc_start) - val[0]
        end_cond = val[1] - int(compare_loc_end)
        
        if (((start_cond > 0 and start_cond <= genomeDist) or (end_cond > 0 and end_cond <= genomeDist)) and (compare_strain == val[2])):
            found_protein = foundProteinLoc_df.loc[ind, ['found_protein']].values[0]
            lst_found_protein.append(found_protein)
        locus_tag = tagsProtein_df.loc[i, ['locus_tag']].values[0]
        locusFoundProtein_dict.update({ locus_tag : ', '.join(lst_found_protein) })
    
    n += 1

In [ ]:
# checking
locusFoundProtein_dict

In [ ]:
### creating data frame of found OG index with minimal distances

locusFoundProtein_df = pd.DataFrame(locusFoundProtein_dict, index=['found_protein']).T
locusFoundProtein_df['locus_tag'] = locusFoundProtein_df.index

locusFoundProtein_df = locusFoundProtein_df.reset_index(drop=True)
locusFoundProtein_df = locusFoundProtein_df.loc[locusFoundProtein_df['found_protein'] != '']

cols = locusFoundProtein_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
locusFoundProtein_df = locusFoundProtein_df[cols]

# checking
locusFoundProtein_df

In [ ]:
### merging of the locusFoundProtein_df and tagsProtein_df data frames

locusFoundProteinProt_df = locusFoundProtein_df.merge(tagsProtein_df[['locus_tag', 'protein', diffExprColName]], how='inner')

# checking 
locusFoundProteinProt_df

In [ ]:
### writing results in file.xlsx and file.csv

locusFoundProtein_df.to_excel((mainPath + 'Results/' + cytokin + '/' + cytokin + '_locus_found_protein_' + str(minimalOGdist) + '_' + str(genomeDist) + '.xlsx'), index=False)
locusFoundProtein_df.to_csv((mainPath + 'Results/' + cytokin + '/' + cytokin + '_locus_found_protein_' + str(minimalOGdist) + '_' + str(genomeDist) + '.csv'), sep='\t', index=False)

In [ ]:
### searching initial proteins in found proteins;
### identification of connected initial genes and creating graph of these connections

G = nx.MultiGraph()
index_lst = locusFoundProteinProt_df.index.to_list()

for ind in index_lst:
    
        compareDiffExpr = locusFoundProteinProt_df.loc[ind][diffExprColName]
        #if compareDiffExpr == '':  
         #   continue
        #else:
        prot = locusFoundProteinProt_df.loc[ind].protein

        
        initialTag = locusFoundProteinProt_df.loc[ind].locus_tag
        if prot != '':
            
            bool_lst = locusFoundProtein_df.found_protein.str.contains(prot, regex=False)
            found_df = locusFoundProtein_df.loc[bool_lst]
            
            if not (found_df.empty):
                                
                tags = found_df.locus_tag.to_list()
                
                for tag in tags:
                    
                    diffExpr = locusFoundProteinProt_df.loc[locusFoundProteinProt_df['locus_tag'] == tag][diffExprColName].values[0]
                    if diffExpr != '' and compareDiffExpr != '':
                        if (compareDiffExpr > 0 and diffExpr > 0) or (compareDiffExpr < 0 and diffExpr < 0):
                            G.add_edge(initialTag, tag)
                    else:
                        G.add_edge(initialTag, tag)               

In [ ]:
### visualisation of the graph and writing results in file.png

pos = nx.spiral_layout(G, equidistant=True, scale=3)  
options = {
    'node_color': 'black',
    'edge_color': 'red',
    'node_size': 0,
    #'linewidths': 0,
    'width': 0.5,
    'with_labels': True,
    'font_size': 10,
    #'font_family': 'cursive'
}
plt.figure(3,figsize=(11, 11), dpi=200) 
nx.draw(G, pos, **options)

plt.savefig(mainPath + 'Results/' + cytokin + '/' + cytokin + '_experimental_partner_for_experimental_tags_' + str(minimalOGdist) + '_' + str(genomeDist) + '.png')

In [ ]:
### converting the graph to data frame

partnerTags_df = nx.to_pandas_edgelist(G).drop_duplicates().groupby('source', as_index=False).aggregate(lambda x: ', '.join(x))
partnerTags_df.columns = ['initial tag', 'found partner tags']

# checking
partnerTags_df.head(50)

In [ ]:
### writing results in file.xlsx and file.csv

partnerTags_df.to_excel((mainPath + 'Results/' + cytokin + '/' + cytokin + '_experimental_partner_for_experimental_tags_' + str(minimalOGdist) + '_' + str(genomeDist) + '.xlsx'), index=False)
partnerTags_df.to_csv((mainPath + 'Results/' + cytokin + '/' + cytokin + '_experimental_partner_for_experimental_tags_' + str(minimalOGdist) + '_' + str(genomeDist) + '.csv'), sep='\t', index=False)